The Australian DEM is 35GB, and hosted by the DEA [here](https://data.dea.ga.gov.au/?prefix=projects/elevation/ga_srtm_dem1sv1_0/)
AWS bucket name to use to access the data is `dea-public-data`

## Original Data Source

*SRTM-derived 1 Second Digital Elevation Models Version 1.0*

The 1 second Shuttle Radar Topography Mission (SRTM) Digital Elevation
Models Version 1.0 package comprises three surface models: the Digital
Elevation Model (DEM), the Smoothed Digital Elevation Model (DEM-S)
and the Hydrologically Enforced Digital Elevation Model (DEM-H).

### sources and guides
[DEA guide to accessing data from their public S3 bucket using STAC](https://docs.dea.ga.gov.au/guides/setup/gis/stac/)

[Downloading and streaming data using STAC metadata](https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/)

In [1]:
import os
import numpy

import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt

import geopandas as gpd
import rasterio
from rasterio.windows import Window, from_bounds
from rasterio import plot

import pystac_client
import odc.stac
from odc.algo import to_rgba
import stackstac

import leafmap.foliumap as leafmap

In [2]:
# #leaving this alone for now as I don't get how to make it interactive. Will use hardcoded file as palceholder.
# uploader = widgets.FileUpload(
#     accept='.geojson',
#     multiple=False
# )
# display(uploader)

In [3]:
# set variables here
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

geom_gpd = gpd.read_file('data/temp/dissolved-boundaries.geojson')
bbox = list(geom_gpd.total_bounds)

catalog_dem = pystac_client.Client.open("https://explorer.sandbox.dea.ga.gov.au/stac/")
catalog_rgb = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

collection_dem = ['ga_srtm_dem1sv1_0']
collection_rgb = ['sentinel-2-l2a']

# start and end date for landsat imagery. This is for making a colour image to compare the DEM to.
start_date = "2023-01-01"
end_date = "2023-01-30"

crs = "EPSG:3857"
resolution = 90 #added to resolve error "Failed to auto-guess CRS/resolution."

print(bbox)

[116.26012130269045, -29.307384715430175, 116.3875862387774, -29.220237788279107]


In [4]:
# Build a query with the set parameters
query_dem = catalog_dem.search(
    bbox=bbox, 
    collections=collection_dem
)

# Search the STAC catalog for all items matching the query
items_dem = list(query_dem.items())
print(f"Found: {len(items_dem):d} datasets")

Found: 1 datasets


In [5]:
ds_dem = odc.stac.stac_load(
    items_dem,
    bands=("dem_h"),
    crs=crs,
    resolution = resolution,
    chunks={},
    groupby="solar_day",
    bbox=bbox,
)
ds_dem

<xarray.Dataset>
Dimensions:      (y: 125, x: 158, time: 1)
Coordinates:
  * y            (y) float64 -3.404e+06 -3.404e+06 ... -3.415e+06 -3.415e+06
  * x            (x) float64 1.294e+07 1.294e+07 ... 1.296e+07 1.296e+07
    spatial_ref  int32 3857
  * time         (time) datetime64[ns] 2014-12-15T14:58:44
Data variables:
    dem_h        (time, y, x) float32 dask.array<chunksize=(1, 125, 158), meta=np.ndarray>

In [7]:
ds_rgb = catalog_rgb.search(bbox=bbox,
                             datetime=f"{start_date}/{end_date}",
                             collections=collection_rgb,
                             max_items = 10)
print(f"Total number of matching items: {ds_rgb.matched()}")

for item in ds_rgb.get_items():
    print(item)
assets = item.assets

Total number of matching items: 4
<Item id=S2A_50JMN_20230126_0_L2A>
<Item id=S2B_50JMN_20230121_0_L2A>
<Item id=S2B_50JMN_20230111_0_L2A>
<Item id=S2B_50JMN_20230101_0_L2A>


In [9]:
assets.plot.imshow(row="time", rgb="band", robust=True, size=6)

AttributeError: 'dict' object has no attribute 'plot'

In [ ]:
with rasterio.open(assets['red'].href) as src:
    red = src.read(1)
    
with rasterio.open(assets['green'].href) as src:
    green = src.read(1)
    
with rasterio.open(assets['blue'].href) as src:
    blue = src.read(1)

In [ ]:
ds_dem.dem_h.plot()

In [ ]:
# Initialize a leafmap map
m = leafmap.Map(height="800px", width="1200px", draw_control=False, measure_control=False, fullscreen_control=False, attribution_control=True)

m.add_layer(ds_dem.dem_h)